In [ ]:
import os
import time

import pandas as pd
import yfinance as yf

# 取得対象のフォルダパス
input_folder = "C:/data/a/"
output_folder = "C:/data/stooq/"

# 出力フォルダが存在しない場合は作成
os.makedirs(output_folder, exist_ok=True)

# フォルダ内のCSVファイルを取得
csv_files = [f for f in os.listdir(input_folder) if f.endswith(".csv")]

# 株価データを取得して保存
for csv_file in csv_files:
    try:
        # CSVを読み込み（1行目を列名として読み込む）
        file_path = os.path.join(input_folder, csv_file)
        df = pd.read_csv(file_path)

        # "ticker" 列からティッカーコードを取得
        if "ticker" not in df.columns:
            print(f"⚠️ {csv_file} に 'ticker' 列が存在しません")
            continue

        # 小数点を消して整数として処理
        ticker_symbol = str(int(df.loc[0, "ticker"])) + ".T"

        # "Unnamed: 0_bs" 列から日付を取得（2行目以降）
        if "Unnamed: 0_bs" not in df.columns:
            print(f"⚠️ {csv_file} に 'Unnamed: 0_bs' 列が存在しません")
            continue

        # 1行目は列名なので2行目以降を取得
        dates = (
            df["Unnamed: 0_bs"].dropna().astype(str).tolist()
        )  # 2行目以降の全ての日付を取得

        print(f"取得した日付リスト: {dates}")  # 日付リストを確認

        # 株価取得用オブジェクト
        ticker = yf.Ticker(ticker_symbol)
        # 株価を格納するリスト（dates と同じ長さで初期化）
        stock_prices = [None] * len(dates)

        # 日付ごとに株価を取得
        for i, date in enumerate(dates):
            try:
                # 株価取得の処理
                price = ticker.history(
                    start=pd.to_datetime(date) - pd.Timedelta(days=10),
                    end=pd.to_datetime(date) + pd.Timedelta(days=1),
                )

                if not price.empty:
                    # タイムゾーンの無効化
                    price.index = price.index.tz_localize(None)

                    # 指定日の株価取得
                    date_obj = pd.to_datetime(date)
                    if date_obj in price.index:
                        stock_price = price.loc[date_obj, "Close"]
                        print(f"📅 {date} の株価: {stock_price}")  # 取得した株価を表示
                    else:
                        # 直前の営業日のデータを取得
                        stock_price = price["Close"].asof(date_obj)
                        print(
                            f"📅 {date} の直前営業日の株価: {stock_price}"
                        )  # 直前の営業日の株価を表示

                    # インデックスを超えないように追加
                    if i < len(stock_prices):
                        stock_prices[i] = stock_price
                    else:
                        print(f"⚠️ インデックス {i} が範囲外です")
                else:
                    print(f"⚠️ {ticker_symbol} の {date} 付近のデータなし")

            except Exception as e:
                print(f"❌ エラー: {ticker_symbol} ({date}) - {e}")
                if i < len(stock_prices):
                    stock_prices[i] = None  # データがない場合は None にする

            time.sleep(1)  # API制限回避のための遅延

        # 株価リストをデータフレームに追加
        # 株価リストをデータフレームに追加
        # stock_pricesの長さをdfの行数に合わせる
        while len(stock_prices) < len(df):
            stock_prices.append(None)  # 足りない分はNoneを追加

        # 株価リストをデータフレームに追加
        df["Stock_Price"] = stock_prices

        # 出力ファイルのパス（上書きしない場合は output_folder に保存）
        output_path = os.path.join(output_folder, csv_file)
        df.to_csv(output_path, encoding="utf-8-sig", index=False)

        print(f"✅ {csv_file} のデータを {output_path} に保存完了")

    except Exception as e:
        print(f"❌ ファイル処理エラー: {csv_file} - {e}")

# すべての処理が完了
print("✅ すべての処理が完了しました")

In [6]:
import os
import time

import pandas as pd
import pandas_datareader.data as web

# 入出力フォルダ
input_folder = "C:/data/output/"
output_folder = "C:/data/stooq/"

# 出力フォルダを作成（存在しない場合）
os.makedirs(output_folder, exist_ok=True)

# フォルダ内のCSVファイルを取得
csv_files = [f for f in os.listdir(input_folder) if f.endswith(".csv")]

# 株価データを取得して保存
for csv_file in csv_files:
    try:
        file_path = os.path.join(input_folder, csv_file)
        df = pd.read_csv(file_path)

        # "ticker" 列の取得
        if "ticker" not in df.columns:
            print(f"⚠️ {csv_file} に 'ticker' 列が存在しません")
            continue
        ticker_symbol = str(int(df.loc[0, "ticker"])) + ".JP"

        # "Unnamed: 0_bs" から日付を取得（2行目以降）
        if "Unnamed: 0_bs" not in df.columns:
            print(f"⚠️ {csv_file} に 'Unnamed: 0_bs' 列が存在しません")
            continue
        dates = df["Unnamed: 0_bs"].dropna().astype(str).tolist()  # 2行目以降

        print(f"📅 {csv_file} の取得日リスト: {dates}")

        # 株価リスト（初期化）
        stock_prices = [None] * len(dates)

        # 日付ごとに株価を取得
        for i, date in enumerate(dates):
            try:
                date_obj = pd.to_datetime(date)

                # Stooq からデータ取得（10日前から取得）
                price = web.DataReader(
                    ticker_symbol,
                    "stooq",
                    start=date_obj - pd.Timedelta(days=10),
                    end=date_obj,
                )

                if not price.empty:
                    price = price.sort_index()  # 日付順に並び替え

                    if date_obj in price.index:
                        stock_price = price.loc[date_obj, "Close"]
                    else:
                        stock_price = price["Close"].asof(date_obj)  # 直前営業日

                    stock_prices[i] = stock_price
                    print(f"✅ {ticker_symbol} {date}: {stock_price}")

                else:
                    print(f"⚠️ {ticker_symbol} の {date} のデータなし")

            except Exception as e:
                print(f"❌ エラー: {ticker_symbol} ({date}) - {e}")
                stock_prices[i] = None

            time.sleep(1)  # API制限回避

        # `stock_prices` の長さを df の行数に調整
        while len(stock_prices) < len(df):
            stock_prices.append(None)  # 足りない分は None で埋める

        # 株価リストをデータフレームに追加
        df["Stock_Price"] = stock_prices

        # 出力ファイルのパス（上書きしない場合は output_folder に保存）
        output_path = os.path.join(output_folder, csv_file)
        df.to_csv(output_path, encoding="utf-8-sig", index=False)

        print(f"✅ {csv_file} のデータを {output_path} に保存完了")

    except Exception as e:
        print(f"❌ ファイル処理エラー: {csv_file} - {e}")

# すべての処理が完了
print("✅ すべての処理が完了しました")

📅 1376_financial.csv の取得日リスト: ['2024-05-31', '2023-05-31', '2022-05-31', '2021-05-31']
⚠️ 1376.JP の 2024-05-31 のデータなし
⚠️ 1376.JP の 2023-05-31 のデータなし
⚠️ 1376.JP の 2022-05-31 のデータなし


KeyboardInterrupt: 

In [ ]:
!pip install pandas-datareader